In [ ]:
%cd /content/
!pip install --upgrade pip
!pip install torch==2.0.1+cu118 torchvision==0.15.2+cu118 --extra-index-url https://download.pytorch.org/whl/cu118

# Installing TinyCuda
%cd /content/
!gdown "https://drive.google.com/u/1/uc?id=1-7x7qQfB7bIw2zV4Lr6-yhvMpjXC84Q5&confirm=t"
!pip install tinycudann-1.7-cp310-cp310-linux_x86_64.whl

# Install nerfstudio
%cd /content/
!pip install git+https://github.com/nerfstudio-project/nerfstudio.git

!git clone https://github.com/alaa-khamis/looping-gifs-nerf.git

# Run COLMAP locally using the colmap2nerf.py file in [Instant-NGP](https://github.com/NVlabs/instant-ngp)
1. Download instant-ngp locally
2. Follow the instructions in this file: [Instant-NGP](https://github.com/NVlabs/instant-ngp/blob/master/docs/nerf_dataset_tips.md)
3. Upload the data to your google drive (images + transform.json)

In [ ]:
#Load Data
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

scene = 'myvid'     #Change to directory containing data

# Open Viewer

In [ ]:

%cd /content

# Install localtunnel
# We are using localtunnel https://github.com/localtunnel/localtunnel but ngrok could also be used
!npm install -g localtunnel

# Tunnel port 7007, the default for
!rm url.txt 2> /dev/null
get_ipython().system_raw('lt --port 7007 >> url.txt 2>&1 &')

import time
time.sleep(3) # the previous command needs time to write to url.txt


with open('url.txt') as f:
  lines = f.readlines()
websocket_url = lines[0].split(": ")[1].strip().replace("https", "wss")
# from nerfstudio.utils.io import load_from_json
# from pathlib import Path
# json_filename = "nerfstudio/nerfstudio/viewer/app/package.json"
# version = load_from_json(Path(json_filename))["version"]
url = f"https://viewer.nerf.studio/?websocket_url={websocket_url}"
print(url)
print("You may need to click Refresh Page after you start training!")
from IPython import display
display.IFrame(src=url, height=800, width="100%")

# Train data (default model is 'nerfacto', other available models : [])

In [ ]:
import os

%cd /content
if os.path.exists(f"drive/MyDrive/{scene}/transforms.json"):
    !ns-train nerfacto --viewer.websocket-port 7007 nerfstudio-data --data drive/MyDrive/$scene
else:
    from IPython.core.display import display, HTML
    display(HTML('Error: Data processing did not complete'))
    display(HTML('Please re-run `Downloading and Processing Data`, or view the FAQ for more info.'))

# Camera Path Creator

This Python script allows you to create a camera path from transformation data. The script uses JSON data as an input, containing frame and transformation data, and creates a camera path JSON file as output.

## Parameters

The script allows you to adjust the following parameters via command-line arguments:

- `--mode`: The aspect ratio of the video. It can be 'landscape' or 'portrait'. Default value is 'landscape'.
- `--fov`: The field of view of the camera. Default value is 50.
- `--aspect-ratio`: The aspect ratio of the video. Default value is 1.66996699669967 (approximately 16:9), which corresponds to widescreen aspect ratio.
- `--height`: The height of the video in pixels. Default value is 1080.
- `--width`: The width of the video in pixels. Default value is 1920.
- `--duration`: The duration of the video in seconds. Default value is 5.
- `--smoothness`: Controls the smoothness of the camera path. Default value is 0.5.
- `--data`: The path to your input JSON file with frame and transformation data. Default value is './transforms.json'.
- `--output`: The directory where you want your output JSON file to be saved. Default value is '.' (current directory).

## How to Use

To run the script with default parameters, use the following command:

```bash
python camera_path_creator.py
```

To run the script with custom parameters, use the following command:

```bash
python camera_path_creator.py --mode portrait --fov 75 --aspect-ratio 1.33 --height 1080 --width 720 --duration 10 --smoothness 0.8 --data ./custom_data.json --output ./output_dir
```

This command will create a portrait video with a FOV of 75, an aspect ratio of 1.33 (4:3), a resolution of 720x1080, a duration of 10 seconds, and a smoothness value of 0.8. The input data is taken from custom_data.json and the output will be saved in the directory output_dir.

In [ ]:
transforms_path = f"drive/MyDrive/{scene}/transforms.json"
output_path = "camera_paths/"

!python create_cam_path.py --load $transforms_path --output $output_path

# Change config_path and camera_path_file to fit your data

In [ ]:

config_path = 'drive/MyDrive/nerfacto/2023-07-22_190553/config.yml'
camera_path_file = 'camera_paths/camera_path.json'
output_path = 'renders/output.mp4'

%cd /content/
!ns-render camera-path --load-config $config_path --camera-path-filename $camera_path_file --output-path $output_path